# Olist Dataset cleaning

In [1]:
# Required package installation
%pip install pandas
%pip install tqdm

# Necessary imports
import os
import pandas as pd
from tqdm.notebook import tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Load datasets
Requires that all the datasets to be in an `./input` folder (relative to the current notebook's path).

In [2]:
import pandas as pd

df_customers = pd.read_csv('./input/olist_customers_dataset.csv')
df_carts = pd.read_csv('./input/olist_order_items_dataset.csv')
df_payments = pd.read_csv('./input/olist_order_payments_dataset.csv')
df_orders = pd.read_csv('./input/olist_orders_dataset.csv')
df_products = pd.read_csv('./input/olist_products_dataset.csv')
df_sellers = pd.read_csv('./input/olist_sellers_dataset.csv')
df_category_name_translations = pd.read_csv('./input/product_category_name_translation.csv')

## Payments
* Remove `payment_sequential` from Payment.
* Remove `order_id` from Payment.
* Add an unique ID to each payment.

In [3]:
df_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [4]:
# Create payment_id series
payment_id = pd.Series(df_payments.index.get_level_values(0).values).apply(lambda x: f'pid{x}')
df_payments['payment_id'] = payment_id

# Drop unused columns
df_payments = df_payments.drop(columns=['payment_sequential'])
df_payments.head()

,order_id,payment_type,payment_installments,payment_value,payment_id
0,b81ef226f3fe1789b1e8b2acac839d17,credit_card,8,99.33,pid0
1,a9810da82917af2d9aefd1278f1dcfa0,credit_card,1,24.39,pid1
2,25e8ea4e93396b6fa0d3dd708e76c1bd,credit_card,1,65.71,pid2
3,ba78997921bbcdc1373bb41e913ab953,credit_card,8,107.78,pid3
4,42fdf880ba16b47b59251dd489d4441a,credit_card,2,128.45,pid4


## Products
* Add 'miscellaneous' category for products with null category.
* Delete columns: `name_length`, `product_descr_length` and `photos_qty`
* Delete rows with nulls in `product_weight_g`, `product_length_cm`, `product_height_cm` or `product_width_cm`, save those `product_id` to remove orders and carts that contain said products.
* Translate category names.

In [5]:
df_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [6]:
# Filter out columns
df_products = df_products[['product_id', 'product_category_name', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']]

# Assign 'miscellaneous' category
selector = df_products['product_category_name'].isnull()
df_products.loc[selector, 'product_category_name'] = 'miscellaneous'

# Drop rows with Nulls in dimension-related information columns
cond_1 = df_products['product_weight_g'].isnull()
cond_2 = df_products['product_length_cm'].isnull()
cond_3 = df_products['product_height_cm'].isnull()
cond_4 = df_products['product_width_cm'].isnull()
df_products_to_delete = df_products[cond_1 | cond_2 | cond_3 | cond_4]
df_products = df_products.drop(df_products_to_delete.index.get_level_values(0))

# Translate category names to english
series_category = df_products['product_category_name']
series_category = series_category.replace(df_category_name_translations['product_category_name'].values, df_category_name_translations['product_category_name_english'].values)
df_products['product_category_name'] = series_category

# Rename columns
df_products = df_products.rename(columns={'product_category_name': 'product_category'})

In [7]:
df_products_to_delete

,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,miscellaneous,NaN,NaN,NaN,NaN


In [8]:
df_products.head()

,product_id,product_category,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,625.0,20.0,17.0,13.0


## Carts
* Remove multiple appearances of same {`order_id`, `product_id`, `seller_id`} and assign a quantity number, representing how many Products `product_id` items were bought from Seller `seller_id` within Order `order_id`.
* Remove carts that contains products from `df_products_to_delete`.

In [9]:
df_carts.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [10]:
# Group carts and count quantity of products bought from same seller within an order
df_carts_counts = df_carts.groupby(['order_id', 'product_id', 'seller_id'])
series_qty = df_carts_counts.count().reset_index()['order_item_id']

# Add the quantity column
df_carts_counts = df_carts_counts.first().reset_index()
df_carts_counts['quantity'] = list(series_qty)

# Select column and rename
df_carts_counts = df_carts_counts[['order_id', 'product_id', 'seller_id', 'quantity', 'price', 'freight_value', 'shipping_limit_date']]
df_carts_counts = df_carts_counts.rename(columns={'freight_value': 'price_freight'})
df_carts_counts.head()

,order_id,product_id,seller_id,quantity,price,price_freight,shipping_limit_date
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,1,58.90,13.29,2017-09-19 09:45:35
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,1,239.90,19.93,2017-05-03 11:05:13
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,1,199.00,17.87,2018-01-18 14:48:30
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,1,12.99,12.79,2018-08-15 10:10:18
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,1,199.90,18.14,2017-02-13 13:57:51


In [11]:
# Find rows with removed products
df_carts_to_be_deleted_1 = df_carts_counts[df_carts_counts['product_id'].isin(df_products_to_delete['product_id'])]
df_carts_to_be_deleted_2 = df_carts_counts[~df_carts_counts['product_id'].isin(df_products['product_id'])]

df_carts_to_be_deleted = pd.concat([df_carts_to_be_deleted_1, df_carts_to_be_deleted_2])
df_carts_to_be_deleted.head()

,order_id,product_id,seller_id,quantity,price,price_freight,shipping_limit_date
6462,101157d4fae1c9fb74a00a5dee265c25,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,1,29.0,14.52,2017-04-11 08:02:26
8429,1521c6bb7b1028154c8c67cf80fa809f,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,1,29.0,16.05,2017-04-07 10:10:16
26064,415cfaaaa8cea49f934470548797fed1,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,2,29.0,14.52,2017-04-07 10:35:19
35677,595316a07cd3dea9db7adfcc7e247ae7,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,1,39.0,9.27,2017-08-18 04:26:04
43977,6e150190fbe04c642a9cf0b80d83ee16,5eb564652db742ff8f28759cd8d2652a,4e922959ae960d389249c378d1c939f5,1,39.0,16.79,2017-06-30 16:45:14


In [12]:
# Drop rows
df_carts_counts = df_carts_counts.drop(df_carts_to_be_deleted.index.get_level_values(0))
df_carts_counts.head()

,order_id,product_id,seller_id,quantity,price,price_freight,shipping_limit_date
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,1,58.90,13.29,2017-09-19 09:45:35
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,1,239.90,19.93,2017-05-03 11:05:13
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,1,199.00,17.87,2018-01-18 14:48:30
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,1,12.99,12.79,2018-08-15 10:10:18
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,1,199.90,18.14,2017-02-13 13:57:51


## Orders
* For orders with Null values on `order_approved_at`, change `status` to `delivered`, set `order_approved_at` to `order_purchase_timestamp`.
* For orders with Null values on `order_delivered_carrier_date` with `status`: `processing`, `approved` or `delivered`, set `order_delivered_carrier_date` to `order_approved_at`.
* For orders with Null values on `order_delivered_customer_date` with `status` as `delivered`, set `order_delivered_customer_date` to `order_delivered_carrier_date`.
* Remove orders with removed products (by carts selected on previous step).

Grouping all orders by `order_status`, then counting the number of nulls in each timestamp-related column:

In [13]:
for status in df_orders.order_status.unique():
    g = df_orders[df_orders.isna().any(axis=1)].groupby('order_status')
    df = df_orders.iloc[g.groups[status]]

    print('status:', status)
    print('rows', len(df))
    print()
    print('nulls:')
    print('order_purchase_timestamp', len(df[df.order_purchase_timestamp.isnull()]))
    print('order_approved_at', len(df[df.order_approved_at.isnull()]))
    print('order_delivered_carrier_date', len(df[df.order_delivered_carrier_date.isnull()]))
    print('order_delivered_customer_date', len(df[df.order_delivered_customer_date.isnull()]))
    print('order_estimated_delivery_date', len(df[df.order_estimated_delivery_date.isnull()]))
    print('\n---------------------------------------\n')

status: delivered
rows 23

nulls:
order_purchase_timestamp 0
order_approved_at 14
order_delivered_carrier_date 2
order_delivered_customer_date 8
order_estimated_delivery_date 0

---------------------------------------

status: invoiced
rows 314

nulls:
order_purchase_timestamp 0
order_approved_at 0
order_delivered_carrier_date 314
order_delivered_customer_date 314
order_estimated_delivery_date 0

---------------------------------------

status: shipped
rows 1107

nulls:
order_purchase_timestamp 0
order_approved_at 0
order_delivered_carrier_date 0
order_delivered_customer_date 1107
order_estimated_delivery_date 0

---------------------------------------

status: processing
rows 301

nulls:
order_purchase_timestamp 0
order_approved_at 0
order_delivered_carrier_date 301
order_delivered_customer_date 301
order_estimated_delivery_date 0

---------------------------------------

status: unavailable
rows 609

nulls:
order_purchase_timestamp 0
order_approved_at 0
order_delivered_carrier_date 6

It shows that there are problems with nulls in orders with `order_status` set to `delivered`, as these orders should have all their timestamp already set. To fill in the missing values, each null value will be replaced with the next non-null value found on the same row, up to `order_estimated_delivery_date`, that always has a value.

For orders with other status, there are no logic-related problems:

* `invoiced`: The payment has been received and approved, so the package has not been sent to the carrier company.
* `shipped`: The package has been shipped but not received by the customer.
* `processing`: The order has been paid, but the system is awaiting confirmation by the seller.
* `unavailable`: An order can be canceled up to an approved status.
* `created`: An order that was just created, it has not been paid yet.
* `approved`: The package is awaiting to be sent to the carrier company.

In the light of these descriptions, an enumeration based on `order_status` can be set:

* 0: `created`
* 1: `approved`
* 2: `processing`
* 3: `invoiced`
* 4: `shipped`
* 5: `delivered`

And, for sake of completion, assigning error-codes to status out of cycle:

* -1: `unavailable`
* -2: `canceled`

Using this enumeration system leads to a diminished database size, and allows to make queries based on ranges of status, such as: "status in between `invoiced` and `delivered`", being traslated as `3 < order_status < 5`.

In [14]:
# For orders with status: delivered
cond_1 = df_orders['order_status'] == 'delivered'

# Set nulls in order_delivered_customer_date to order_estimated_delivery_date
cond_2 = df_orders['order_delivered_customer_date'].isnull()
df_orders.loc[cond_1 & cond_2, 'order_delivered_customer_date'] = df_orders[cond_1 & cond_2]['order_estimated_delivery_date']

# Set nulls in order_delivered_carrier_date to order_delivered_customer_date
cond_2 = df_orders['order_delivered_carrier_date'].isnull()
df_orders.loc[cond_1 & cond_2, 'order_delivered_carrier_date'] = df_orders[cond_1 & cond_2]['order_delivered_customer_date']

# Set nulls in order_approved_at to order_delivered_carrier_date
cond_2 = df_orders['order_approved_at'].isnull()
df_orders.loc[cond_1 & cond_2, 'order_approved_at'] = df_orders[cond_1 & cond_2]['order_delivered_carrier_date']

In [15]:
# Remove orders from deleted carts
df_orders = df_orders[~df_orders['order_id'].isin(df_carts_to_be_deleted['order_id'])]
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [16]:
# Change order_status into enumeration form
order_status_map = {
    'created': 0,
    'approved': 1,
    'processing': 2,
    'invoiced': 3,
    'shipped': 4,
    'delivered': 5,
    'unavailable': -1,
    'canceled': -2
}

series_status_enum = df_orders['order_status'].apply(lambda s: order_status_map[s])
df_orders = df_orders.drop(columns=['order_status'])
df_orders['order_status'] = series_status_enum
df_orders.head()

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,5
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,5
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,5
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,5
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,5


## Address
* Retrieve all address from customers and sellers.
* Compose a DataFrame out of it.

In [17]:
# Group customers by unique customers, and assign first address 
df_unique_customers = df_customers.groupby('customer_unique_id').first().reset_index()

# Create address_id series for customers
customer_address_id = pd.Series(df_unique_customers.index.get_level_values(0).values).apply(lambda x: f'aid_c{x}')
df_unique_customers['address_id'] = customer_address_id

df_unique_customers.head()

,customer_unique_id,customer_id,customer_zip_code_prefix,customer_city,customer_state,address_id
0,0000366f3b9a7992bf8c76cfdf3221e2,fadbb3709178fc513abc1b2670aa1ad2,7787,cajamar,SP,aid_c0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4cb282e167ae9234755102258dd52ee8,6053,osasco,SP,aid_c1
2,0000f46a3911fa3c0805444483337064,9b3932a6253894a02c1df9d19004239f,88115,sao jose,SC,aid_c2
3,0000f6ccb0745a6a4b88665a16c9f078,914991f0c02ef0843c0e7010c819d642,66812,belem,PA,aid_c3
4,0004aac84e0df4da2b147fca70cf8255,47227568b10f5f58a524a75507e6992c,18040,sorocaba,SP,aid_c4


In [18]:
# Normalize columns names within customers addresses
df_address_customers_normalized = df_unique_customers[['address_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']]
df_address_customers_normalized = df_address_customers_normalized.rename(columns={'customer_zip_code_prefix': 'zip_code_prefix',
                                                                                  'customer_city': 'city',
                                                                                  'customer_state': 'state'})
df_address_customers_normalized.head()

,address_id,zip_code_prefix,city,state
0,aid_c0,7787,cajamar,SP
1,aid_c1,6053,osasco,SP
2,aid_c2,88115,sao jose,SC
3,aid_c3,66812,belem,PA
4,aid_c4,18040,sorocaba,SP


In [19]:
# Set Address Dataframe to just use customer address data
df_address = df_address_customers_normalized
df_address.head()

,address_id,zip_code_prefix,city,state
0,aid_c0,7787,cajamar,SP
1,aid_c1,6053,osasco,SP
2,aid_c2,88115,sao jose,SC
3,aid_c3,66812,belem,PA
4,aid_c4,18040,sorocaba,SP


In [20]:
# Map from customer_id to customer_unique_id
cid_to_cuid = dict()
for index, row in tqdm(df_customers.iterrows(), total=len(df_customers), desc='Mapping C_id to C_uid'):
    cid_to_cuid[row['customer_id']] = row['customer_unique_id']
    
# Map from customer_unique_id to address_id
cuid_to_aid = dict()
for index, row in tqdm(df_unique_customers.iterrows(), total=len(df_unique_customers), desc='Mapping C_uid to A_id'):
    cuid_to_aid[row['customer_unique_id']] = row['address_id']
    
# Map from customer_id to address_id
cid_to_aid = dict()
for cid in tqdm(cid_to_cuid.keys(), total=len(cid_to_cuid), desc='Mapping C_id to A_id'):
    cuid = cid_to_cuid[cid]
    cid_to_aid[cid] = cuid_to_aid[cuid]

Mapping C_id to C_uid:   0%|          | 0/99441 [00:00<?, ?it/s]

Mapping C_uid to A_id:   0%|          | 0/96096 [00:00<?, ?it/s]

Mapping C_id to A_id:   0%|          | 0/99441 [00:00<?, ?it/s]

In [21]:
# Insert address_id into Orders
series_aid = pd.Series([cid_to_aid[cid] for cid in df_orders['customer_id']])
df_orders['address_id'] = series_aid

print(f'Orders        : {len(df_orders)}')
print(f'Null addresses: {len(df_orders[df_orders.address_id.isnull()])}')

Orders        : 99425
Null addresses: 16


In [22]:
# Drop rows with null address_id from Orders
df_orders_null_aid = df_orders[df_orders.address_id.isnull()]
df_orders = df_orders[~df_orders.address_id.isnull()]

# And Carts associated with those Orders
df_carts_counts = df_carts_counts[~df_carts_counts['order_id'].isin(df_orders_null_aid['order_id'])].copy()

In [23]:
# Fix duplicate city names
selector = df_address['city'] == 'arraial d ajuda'
df_address.loc[selector, 'city'] = 'arraial d\'ajuda'

selector = df_address['city'] == 'embu'
df_address.loc[selector, 'city'] = 'embu das artes'


selector = df_address['city'] == 'mogi-mirim'
df_address.loc[selector, 'city'] = 'mogi mirim'

In [24]:
# Set all city names to lowercase, all state abbreviations to uppercase
df_address['city'] = df_address['city'].str.lower()
df_address['state'] = df_address['state'].str.upper()

## Type casting

In [25]:
# For Payments, all types are correct
df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_type          103886 non-null  object 
 2   payment_installments  103886 non-null  int64  
 3   payment_value         103886 non-null  float64
 4   payment_id            103886 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.0+ MB


In [26]:
# For Orders, timestamps should be casted
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99409 entries, 0 to 99424
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99409 non-null  object
 1   customer_id                    99409 non-null  object
 2   order_purchase_timestamp       99409 non-null  object
 3   order_approved_at              99263 non-null  object
 4   order_delivered_carrier_date   97628 non-null  object
 5   order_delivered_customer_date  96452 non-null  object
 6   order_estimated_delivery_date  99409 non-null  object
 7   order_status                   99409 non-null  int64 
 8   address_id                     99409 non-null  object
dtypes: int64(1), object(8)
memory usage: 7.6+ MB


In [27]:
# Casting timestamps
time_format = '%Y-%m-%d %H:%M:%S'
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'], format=time_format)
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'], format=time_format)
df_orders['order_delivered_carrier_date'] = pd.to_datetime(df_orders['order_delivered_carrier_date'], format=time_format)
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'], format=time_format)
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'], format=time_format)

In [28]:
# For Address, all columns are OK.
df_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96096 entries, 0 to 96095
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   address_id       96096 non-null  object
 1   zip_code_prefix  96096 non-null  int64 
 2   city             96096 non-null  object
 3   state            96096 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.9+ MB


In [29]:
# For Products, dimension-related columns should be casted
df_products['product_weight_g'] = df_products['product_weight_g'].astype(int)
df_products['product_length_cm'] = df_products['product_length_cm'].astype(int)
df_products['product_height_cm'] = df_products['product_height_cm'].astype(int)
df_products['product_width_cm'] = df_products['product_width_cm'].astype(int)
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32949 entries, 0 to 32950
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   product_id         32949 non-null  object
 1   product_category   32949 non-null  object
 2   product_weight_g   32949 non-null  int32 
 3   product_length_cm  32949 non-null  int32 
 4   product_height_cm  32949 non-null  int32 
 5   product_width_cm   32949 non-null  int32 
dtypes: int32(4), object(2)
memory usage: 1.3+ MB


In [30]:
# For Cart, shipping_limit_date should be casted
df_carts_counts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102392 entries, 0 to 102424
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             102392 non-null  object 
 1   product_id           102392 non-null  object 
 2   seller_id            102392 non-null  object 
 3   quantity             102392 non-null  int64  
 4   price                102392 non-null  float64
 5   price_freight        102392 non-null  float64
 6   shipping_limit_date  102392 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 6.2+ MB


In [31]:
# Casting shipping_limit_date
time_format = '%Y-%m-%d %H:%M:%S'
df_carts_counts['shipping_limit_date'] = pd.to_datetime(df_carts_counts['shipping_limit_date'], format=time_format)

## Reorder columns

In [32]:
# Payment
df_payments = df_payments[['payment_id',
                           'order_id',
                           'payment_type',
                           'payment_installments',
                           'payment_value'
                          ]]

df_payments.head()

,payment_id,order_id,payment_type,payment_installments,payment_value
0,pid0,b81ef226f3fe1789b1e8b2acac839d17,credit_card,8,99.33
1,pid1,a9810da82917af2d9aefd1278f1dcfa0,credit_card,1,24.39
2,pid2,25e8ea4e93396b6fa0d3dd708e76c1bd,credit_card,1,65.71
3,pid3,ba78997921bbcdc1373bb41e913ab953,credit_card,8,107.78
4,pid4,42fdf880ba16b47b59251dd489d4441a,credit_card,2,128.45


In [33]:
# Orders
df_orders = df_orders[['order_id',
                       'customer_id',
                       'address_id',
                       'order_status',
                       'order_purchase_timestamp',
                       'order_approved_at',
                       'order_delivered_carrier_date',
                       'order_delivered_customer_date',
                       'order_estimated_delivery_date'
                      ]]

df_orders.head()

,order_id,customer_id,address_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,aid_c46728,5,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,aid_c65829,5,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,aid_c22015,5,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,aid_c46675,5,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,aid_c43056,5,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [34]:
# Address
# Also sorts alphabetically by city name
df_address = df_address.sort_values(by='city')
df_address

,address_id,zip_code_prefix,city,state
66164,aid_c66164,38540,abadia dos dourados,MG
84922,aid_c84922,38540,abadia dos dourados,MG
37982,aid_c37982,38540,abadia dos dourados,MG
53031,aid_c53031,72940,abadiania,GO
75967,aid_c75967,35620,abaete,MG
...,...,...,...,...
75629,aid_c75629,65365,ze doca,MA
63379,aid_c63379,65365,ze doca,MA
79939,aid_c79939,65365,ze doca,MA
80376,aid_c80376,89633,zortea,SC


In [35]:
# Cart
df_carts_counts = df_carts_counts[['seller_id',
                                   'product_id',
                                   'order_id',
                                   'quantity',
                                   'price',
                                   'price_freight',
                                   'shipping_limit_date'
                                   ]]
df_carts_counts.head()

,seller_id,product_id,order_id,quantity,price,price_freight,shipping_limit_date
0,48436dade18ac8b2bce089ec2a041202,4244733e06e7ecb4970a6e2683c13e61,00010242fe8c5a6d1ba2dd792cb16214,1,58.90,13.29,2017-09-19 09:45:35
1,dd7ddc04e1b6c2c614352b383efe2d36,e5f2d52b802189ee658865ca93d83a8f,00018f77f2f0320c557190d7a144bdd3,1,239.90,19.93,2017-05-03 11:05:13
2,5b51032eddd242adc84c38acab88f23d,c777355d18b72b67abbeef9df44fd0fd,000229ec398224ef6ca0657da4fc703e,1,199.00,17.87,2018-01-18 14:48:30
3,9d7a1d34a5052409006425275ba1c2b4,7634da152a4610f1595efa32f14722fc,00024acbcdf0a6daa1e931b038114c75,1,12.99,12.79,2018-08-15 10:10:18
4,df560393f3a51e74553ab94004ba5c87,ac6c3623068f30de03045865e4e10089,00042b26cf59d7ce69dfabb4e55b4fd9,1,199.90,18.14,2017-02-13 13:57:51


In [36]:
# Product
# Also sorts alphabetically by product category
df_products = df_products.sort_values(by='product_category')
df_products

,product_id,product_category,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8441,24fee4a800146a47846fa0e345b6d6ed,agro_industry_and_commerce,800,20,20,20
20819,ea1d59339533cb7f68f0319501d8e4d4,agro_industry_and_commerce,9900,29,49,36
22035,a6bfd5412c292e6ab465ba7d7f794727,agro_industry_and_commerce,200,16,2,20
23918,66e94c5c07d64d44c16cd6903a79a453,agro_industry_and_commerce,12700,56,32,32
6833,163a503e8aa4c48bfd077f416e4341d4,agro_industry_and_commerce,2650,52,13,32
...,...,...,...,...,...,...
19677,ffd2365fb8224dc66883df9351d65deb,watches_gifts,333,16,12,13
6632,ff55834c0b3272853dfe62340f2bb91b,watches_gifts,250,16,2,11
19717,4509f2dcb7651bc7c7fab76f3a87e700,watches_gifts,900,30,10,30
19656,86ba3443f971ab0ff1381a973274f2a5,watches_gifts,300,18,13,16


## Output

In [37]:
# Make sure output folder has been created
try:
    os.mkdir('output')
except:
    pass

df_payments.to_csv('./output/Payments.csv', index=False)
df_orders.to_csv('./output/Orders.csv', index=False)
df_address.to_csv('./output/Addresses.csv', index=False)
df_carts_counts.to_csv('./output/Carts.csv', index=False)
df_products.to_csv('./output/Products.csv', index=False)